# Set up

Define work location :

In [ ]:
project_path = "/Users/victor/Documents/OPENCLASSROOMS/projet_7"

Python librairies imports :

In [ ]:
# File system management
import os

# Data manipulation
import numpy as np
import pandas as pd

# Data algoritms
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

from imblearn.over_sampling import SMOTE

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_theme(palette="Set1")

Load data :

In [ ]:
# Load data
data = pd.read_csv(project_path + "/data/processed/train_features.csv", index_col=[0])

# Functions

# Data preparation

In [ ]:
data = data.reset_index()

# Define target and features
target = data['TARGET']
features = data.drop(columns='TARGET')

In [ ]:
# Check percentage of missing values
print(data.isna().sum().sum()/data.size*100)

In [ ]:
# Create imputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

# Fit to data
imp_mean.fit(features)

# Transform data
data = imp_mean.transform(features)

In [ ]:
# Check class weights
display(target.value_counts())

In [ ]:
# Split dataset
x_train, x_test, y_train, y_test = train_test_split(features, target, train_size=0.8, random_state=0)

In [ ]:
# # Balance data
# sm_data_x, sm_data_y = SMOTE(random_state=0).fit_resample(x_train, y_train)
# sm_data_x = pd.DataFrame(data=sm_data_x, columns=features.columns)
# sm_data_y = pd.DataFrame(data=sm_data_y, columns=['TARGET'])

# # Check balanced data
# print("length of oversampled data is ",len(sm_data_x))
# print("Number of no subscription in oversampled data",len(sm_data_y[sm_data_y['TARGET']==1]))
# print("Number of subscription",len(sm_data_y[sm_data_y['TARGET']==0]))
# print("Proportion of no subscription data in oversampled data is ",len(sm_data_y[sm_data_y['TARGET']==1])/len(sm_data_x))
# print("Proportion of subscription data in oversampled data is ",len(sm_data_y[sm_data_y['TARGET']==0])/len(sm_data_x))

# Classification

In [ ]:
# Create logistic regression model
clf = LogisticRegression(random_state=0)

# Fit to data
clf.fit(x_train, y_train)